In [1]:
import pandas as pd
import geopandas as gpd
import collections
import re
import time
import urllib
import itertools

import numpy as np
import pyodbc

import sqlalchemy
import urllib3

import folium


def get_sql_connection(database_name=None):
    conn_string = "Driver={ODBC Driver 17 for SQL Server};Server=NE-SQL-CL1-03.kf-ext.com;Trusted_Connection=yes"

    if database_name is not None:
        conn_string = conn_string + f";Database={database_name}"

    quote = urllib.parse.quote_plus(conn_string)

    return sqlalchemy.create_engine(f'mssql+pyodbc:///?odbc_connect={quote}')

engine = get_sql_connection()


In [12]:
sql = """SELECT
  g.UDPRN,
  a.UPRN,
  a.[POSTCODE],
  a.[STREET_DESCRIPTION],
  a.[X_COORDINATE] as x,
  a.[Y_COORDINATE] as y,
  a.[LATITUDE] as lat,
  a.[LONGITUDE] as lng,
  a.[ENTRY_DATE],
  a.[STATE_DATE],
  a.[LA_START_DATE],
  a.[RM_START_DATE]

FROM [GeoData].[dbo].[KF_TEMP_HomesEngland_230116_BHL_webdata_geometry] p

join [GeoData].[dbo].[OSAddressBasePlus_Spatial] g
on p.geometry.MakeValid().STCentroid().STBuffer(2000).STContains(g.Geom) = 1

join [GeoData].[dbo].[OSAddressBasePlus_Master] a
on a.UPRN = g.UPRN

join [GeoData].[dbo].[OSAddressBasePlus_ClassLkUp] c
on a.CLASS = c.Concatenated

where a.[CLASS] in ('RD02', 'RD03', 'RD04', 'RD06')

and g.UDPRN <> 0

and a.STATE = 2 --> This means the property is "in use" see STATES codes

and p.Scheme = 'Scheme18'
"""

data = pd.read_sql(sql, engine)
date_cols = ['STATE_DATE', 'ENTRY_DATE', 'RM_START_DATE', 'LA_START_DATE']
for col in date_cols:
	data[col] = pd.to_datetime(data[col])

data['addr_date'] = [min(i) for i in data[date_cols].values]

data = data.sort_values(['UDPRN'])
data = data.drop_duplicates(subset=['UDPRN'], keep='first')

data = data[data['addr_date']>='2012-01-01']

data = data.drop(columns=date_cols)

data = data.reset_index(drop=True)

data

,UDPRN,UPRN,POSTCODE,STREET_DESCRIPTION,x,y,lat,lng,addr_date
0,10314712,10090973062,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
1,10314713,10090973064,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
2,10314714,10090973065,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
3,10314715,10090973066,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
4,10314716,10090973068,GU31 4AW,READON CLOSE,475167.46875,123746.570312,51.008198,-0.929962,2012-03-19
...,...,...,...,...,...,...,...,...,...
509,56694878,10096346855,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
510,56694879,10096346856,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
511,56694880,10096346857,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17
512,56694881,10096346858,GU31 4GF,DRAGON STREET,474810.00000,123181.218750,51.003159,-0.935172,2022-06-17


In [13]:
data.to_csv("property_locations.csv", index=False)